In [1]:
import os
os.chdir("/Users/liutianlin/Desktop/Academics/MINDS/neuromorphic")

import numpy as np
import matplotlib.pyplot as plt
from dynapseReservoir.networks.inputsParameters import in_5_0_P
from dynapseReservoir.networks.reservoirsParameters import generalNetP 
import DYNAPSETools.dynapseSpikesGenerator as DSG


# fix a random seed
np.random.seed(in_5_0_P["seed"]) 

plt.close("all")

In [2]:
# This event indicates the starting time for the experiment
startTrigger = DSG.InputPattern(name = "startTrigger", isiBase = in_5_0_P["isiBase"])
U, C, N = generalNetP["inStartTriggerPos"]
startTrigger.single_event(virtualSourceCoreId = C,
                          neuronAddress = N,
                          coreDest = in_5_0_P["coreDest"],
                          firePeriod = in_5_0_P["startDelay"])


# the stop of experiment
stopTrigger = DSG.InputPattern(name = "stopTrigger", isiBase = in_5_0_P["isiBase"])
U, C, N = generalNetP["inStopTriggerPos"]
stopTrigger.single_event(virtualSourceCoreId = C,
                         neuronAddress = N,
                         coreDest = in_5_0_P["coreDest"],
                         firePeriod = in_5_0_P["stopDelay"])




###  Load in the input

We first write input signals as continuous singal, and convert the continuous signal into spikes


In [3]:
inputMat = np.loadtxt('/Users/liutianlin/Desktop/Academics/MINDS/neuromorphic/reservoir_Demo/reservoirTraining/dynapse_input.txt')

In [4]:

tStart = 0
tStop = 30 # lasts for 10 seconds

all_inputNeuronList = []
all_inputCoreList= []
all_ts_list= []

# a part = 64 neurons. a core = 4 parts. Reservoir = 3 cores.


for core in [1, 2, 3]:
    print('core', core)
    for part in [1,2,3,4]:
        print('part', part)

        neuronInThisPart = np.arange((core-1)*256 + (part-1)*64, (core-1)*256 + (part)*64)

        counter = 0 # count the number of neuron starting from this part

        print('encoding the neuron nr ', (core-1)*256 + (part-1)*64, 'to',  (core-1)*256 + (part)*64 - 1)
        listOfPatterns = []

        for j in neuronInThisPart: 

            neuron_input_t = np.linspace(tStart, tStop, inputMat.shape[1])
            neuron_input_y = inputMat[j, :]

            input_event = DSG.InputPattern(name = "input_event", isiBase = in_5_0_P["isiBase"])

            input_event.threshold_encoder(virtualSourceCoreId = 0,
                                      neuronAddressUpCH = counter + 128,  
                                      neuronAddressDwCH =  counter + 192,  
                                      coreDest = in_5_0_P["coreDest"],   
                                      threshold = 0.5,    
                                      t = neuron_input_t,    
                                      y = neuron_input_y,    
                                      noiseVar = in_5_0_P["sig1Jitter"],      
                                      initDelay = in_5_0_P["sig1Delay"])  


            listOfPatterns.append(input_event)
            counter += 1


        tupleList = []

        for pattern in listOfPatterns:
            prevTime = 0
            patternTuple = []
            for event in pattern.eventList:
                patternTuple.append([event.virtualSourceCoreId, event.neuronAddress, event.coreDest, event.time + prevTime])
                prevTime += event.time
            tupleList.extend(patternTuple)   

        tupleList.sort(key=lambda myList: myList[3]) # sort based on absolute time     


        diffList = [0]*len(tupleList)
        diffList[0] = tupleList[0][3]
        for i in np.arange(1, len(tupleList)):
            diff = tupleList[i][3] - tupleList[i-1][3]
            if diff <= 10:
                diff = 10
            diffList[i] = diff # convert absolute time back to relative time

        mergedPattern = DSG.InputPattern(name = "mergedPattern", isiBase = in_5_0_P["isiBase"])

        for i in np.arange(len(tupleList)):
            virtualSourceCoreId = tupleList[i][0]
            neuronAddress = tupleList[i][1]
            coreDest = tupleList[i][2]
            time = diffList[i]/ 1e5 # change back to second
            mergedPattern.single_event(virtualSourceCoreId, neuronAddress, coreDest, firePeriod = time)


        # ====================== Write events to output file in cAER folder

        # the output file that reford the spikes
        fileName = "/Users/liutianlin/Desktop/Academics/MINDS/neuromorphic/caer/myStimulus/train/" + 'core' + str(core) + '_' + 'part' + str(part) + ".txt"


        DSG.write_to_file(*[startTrigger, mergedPattern, stopTrigger], fileName = fileName)

        
        
        
        
        inputNeuronList = []
        inputCoreList = []
        for item in tupleList:
            if (item[1] >= 128) and (item[1] <= 191) : # this is an exitatory neuron
                inputNeuronList.append((part-1)*64 + item[1] - 128)
                inputCoreList.append( core)
            elif (item[1] >= 192) and (item[1] <= 255) :  # this is an inhi neuron
                inputNeuronList.append((part-1)*64 + item[1] - 192)
                inputCoreList.append(-core)
            else:
                print('sth is wrong!')

#         inputCoreList = [core] * len(inputNeuronList) 
        ts_list = (np.array(np.cumsum(diffList)) - np.array(np.cumsum(diffList))[0]) * 10        
        
        all_inputNeuronList.extend(inputNeuronList)
        all_inputCoreList.extend(inputCoreList)
        all_ts_list.extend(ts_list)


core 1
part 1
encoding the neuron nr  0 to 63
Checking and writing pattern startTrigger
Current pattern lenght is 1
Cumulative pattern lenght is 1. Remaining 32766 events
Pattern startTrigger succesfully written

Checking and writing pattern mergedPattern
Current pattern lenght is 23757
Cumulative pattern lenght is 23758. Remaining 9009 events
Pattern mergedPattern succesfully written

Checking and writing pattern stopTrigger
Current pattern lenght is 1
Cumulative pattern lenght is 23759. Remaining 9008 events
Pattern stopTrigger succesfully written

part 2
encoding the neuron nr  64 to 127
Checking and writing pattern startTrigger
Current pattern lenght is 1
Cumulative pattern lenght is 1. Remaining 32766 events
Pattern startTrigger succesfully written

Checking and writing pattern mergedPattern
Current pattern lenght is 28184
Cumulative pattern lenght is 28185. Remaining 4582 events
Pattern mergedPattern succesfully written

Checking and writing pattern stopTrigger
Current pattern le

In [5]:
len(all_inputNeuronList)

288990

In [6]:
len(all_inputCoreList)

288990

In [7]:
all_ts_list[-1]

30646410.0

In [8]:
inputSpikesAddress = '/Users/liutianlin/Desktop/Academics/MINDS/neuromorphic/reservoir_Demo/reservoirTraining/inputSpikes/'




In [9]:
np.savetxt(inputSpikesAddress + 'input_core_id.txt', all_inputCoreList, fmt='%i')
np.savetxt(inputSpikesAddress + 'input_neuron_id.txt', all_inputNeuronList, fmt='%i')
np.savetxt(inputSpikesAddress + 'input_ts.txt', all_ts_list, fmt='%i')   

In [22]:

tStart = 0
tStop = 30 # lasts for 10 seconds


# a part = 64 neurons. a core = 4 parts. Reservoir = 3 cores.


core = 1
print('core', core)
part = 1
print('part', part)

neuronInThisPart = np.arange((core-1)*256 + (part-1)*64, (core-1)*256 + (part)*64)

counter = 0 # count the number of neuron starting from this part

print('encoding the neuron nr ', (core-1)*256 + (part-1)*64, 'to',  (core-1)*256 + (part)*64 - 1)
listOfPatterns = []

for j in neuronInThisPart: 

    neuron_input_t = np.linspace(tStart, tStop, inputMat.shape[1])
    neuron_input_y = inputMat[j, :]

    input_event = DSG.InputPattern(name = "input_event", isiBase = in_5_0_P["isiBase"])

    input_event.threshold_encoder(virtualSourceCoreId = 0,
                              neuronAddressUpCH = counter + 128,  
                              neuronAddressDwCH =  counter + 192,  
                              coreDest = in_5_0_P["coreDest"],   
                              threshold = 0.5,    
                              t = neuron_input_t,    
                              y = neuron_input_y,    
                              noiseVar = in_5_0_P["sig1Jitter"],      
                              initDelay = in_5_0_P["sig1Delay"])  


    listOfPatterns.append(input_event)
    counter += 1


tupleList = []

for pattern in listOfPatterns:
    prevTime = 0
    patternTuple = []
    for event in pattern.eventList:
        patternTuple.append([event.virtualSourceCoreId, event.neuronAddress, event.coreDest, event.time + prevTime])
        prevTime += event.time
    tupleList.extend(patternTuple)   

tupleList.sort(key=lambda myList: myList[3]) # sort based on absolute time     


diffList = [0]*len(tupleList)
diffList[0] = tupleList[0][3]
for i in np.arange(1, len(tupleList)):
    diff = tupleList[i][3] - tupleList[i-1][3]
    if diff <= 10:
        diff = 10
    diffList[i] = diff # convert absolute time back to relative time

mergedPattern = DSG.InputPattern(name = "mergedPattern", isiBase = in_5_0_P["isiBase"])

for i in np.arange(len(tupleList)):
    virtualSourceCoreId = tupleList[i][0]
    neuronAddress = tupleList[i][1]
    coreDest = tupleList[i][2]
    time = diffList[i]/ 1e5 # change back to second
    mergedPattern.single_event(virtualSourceCoreId, neuronAddress, coreDest, firePeriod = time)

    
    
    
inputNeuronList = []
for item in tupleList:
    if (item[1] >= 128) and (item[1] <= 191) : # this is an exitatory neuron
        inputNeuronList.append((part-1)*64 + item[1] - 128)
    elif (item[1] >= 192) and (item[1] <= 255) :  # this is an inhi neuron
        inputNeuronList.append((part-1)*64 + item[1] - 192)
    else:
        print('sth is wrong!')

inputCoreList = [core] * len(inputNeuronList) 
ts_list = (np.array(np.cumsum(diffList)) - np.array(np.cumsum(diffList))[0]) * 10        

core 1
part 1
encoding the neuron nr  0 to 63


In [25]:
len(inputCoreList)

23757

In [26]:
len(inputCoreList)

23757

In [38]:
ts_list

array([0.000000e+00, 1.000000e+02, 2.000000e+02, ..., 3.062645e+07,
       3.062741e+07, 3.064641e+07])

In [42]:
1e-4

0.0001

In [41]:
1.000000e+02

100.0